### Setting up environment

In [ ]:
# Free of charge tier offers T4 GPUs with 16GB VRAM
# For a 7B size LLM how much memory is needed for inference and for training? What extra data is stored while training?
# Following bitsandbytes and peft libraries with QLORA methods enables finetuning an LLM with 16 GB GPU. How?
! pip install datasets trl ninja packaging bitsandbytes transformers peft accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [ ]:
import torch
import os
import sys
import json
import IPython
import pandas as pd
from datetime import datetime
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    AutoTokenizer,
    TrainingArguments,
)
from trl import SFTTrainer
from huggingface_hub import notebook_login

In [ ]:
# Login to huggingface hub with a write permission token to upload and not to lose your final work.
notebook_login()

### Defining configurations

In [ ]:
# "Generally" any model up to 7B parameter size can be finetuned using the following technique.
# However adjusting configuration maybe needed according to the architecture of model or type of tokenizer.
# The following model is copy of Mistral-7B-v0.1 (https://arxiv.org/abs/2310.06825). It is optimized for uploading on a free-tier colab.
# This model is a base model, meaning it is only pretrained. It is not finetuned for following instructions or human preferences.
# What are the advantages and disadvantages between choosing a base model or a finetuned model for supervised finetuning?
model_name = "cemuluoglakci/Mistral-7B-v0.1_optimized"
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

# For padding we chose "unknow token" id. What could be the other alternatives? What are the advantages and disadvantages between alternatives?
tokenizer.pad_token = tokenizer.unk_token
tokenizer.pad_token_id = tokenizer.unk_token_id

device = 'cuda'

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/918 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [ ]:
# For more information about quantization configuration refer to the following blogpost: "https://huggingface.co/blog/4bit-transformers-bitsandbytes"

bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type="nf4",
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
)

model = AutoModelForCausalLM.from_pretrained(
          model_name,
          quantization_config=bnb_config,
          use_flash_attention_2 = False, # When ever you can access GPUs with newer architecture, use flash attention!
          device_map={"": 0},

)

In [ ]:
# Inspect the layers of the model. What is the effect of quantization?
print(model)

In [ ]:
# For more information about about PEFT configuration, refer to: https://huggingface.co/docs/trl/en/lora_tuning_peft and https://huggingface.co/docs/peft/en/developer_guides/quantization

peft_config = LoraConfig(
        r=16,
        lora_alpha=16,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
        target_modules= ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj", "lm_head",]
)

model = prepare_model_for_kbit_training(model)
model.config.pad_token_id = tokenizer.pad_token_id
model.config.use_cache = False

In [ ]:
# Training argumets will be passed to SFTTrainer wrapper. Wrappers and abstractions make coding easier but debugging harder.
# Check the source code and try to understand what it does:
#    * https://github.com/huggingface/transformers/blob/main/src/transformers/training_args.py
#    * https://github.com/huggingface/trl/blob/main/trl/trainer/sft_trainer.py

# Which parameter is the most critical for the model's performance?
# Which parameters can be improved if you don't have computational limitations?
training_arguments = TrainingArguments(
        output_dir="./results",
        evaluation_strategy="epoch",
        optim="paged_adamw_8bit",
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        gradient_accumulation_steps=1,
        log_level="debug",
        save_steps=500,
        logging_steps=20,
        learning_rate=4e-4,
        num_train_epochs=1,
        warmup_steps=100,
        lr_scheduler_type="constant",
)

### Preparing the dataset

In [ ]:
# Ideally it is better to use all available data for training in multiple epochs. We will use a subsample of the dataset due to computational limitations.
# It is tested that with a fresh google account, more than 600 of “HypoTermQA Evaluation” samples can be used to fine-tune on a free tier account.

SEED = 42
train_sample_count = 300
test_sample_count = 12

In [ ]:
def subsample(dataset, sample_count):
  df = pd.DataFrame(dataset)
  stratify_criteria = df[['IsHypotheticalTerm', 'acceptance_label']]
  _, sample_df = train_test_split(df, test_size=sample_count, stratify=stratify_criteria, random_state=SEED)
  return Dataset.from_pandas(sample_df.drop(columns=["__index_level_0__"]))

In [ ]:
dataset_name = "cemuluoglakci/hallucination_acceptance_agent_instruction_dataset"

In [ ]:
dataset = load_dataset(dataset_name)

dataset['train'] = subsample(dataset['train'], train_sample_count)
dataset['validation'] = subsample(dataset['validation'], test_sample_count)
dataset['test'] = subsample(dataset['test'], test_sample_count)

Generating train split:   0%|          | 0/2988 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/996 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/996 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'acceptance_label', 'isHypotheticalQuestion', 'IsHypotheticalTerm', '__index_level_0__'],
        num_rows: 2988
    })
    validation: Dataset({
        features: ['text', 'acceptance_label', 'isHypotheticalQuestion', 'IsHypotheticalTerm', '__index_level_0__'],
        num_rows: 996
    })
    test: Dataset({
        features: ['text', 'acceptance_label', 'isHypotheticalQuestion', 'IsHypotheticalTerm', '__index_level_0__'],
        num_rows: 996
    })
})

In [ ]:
# Defining trainer with our dataset and configuration
trainer = SFTTrainer(
        model=model,
        train_dataset=dataset['train'],
        eval_dataset=dataset['validation'],
        peft_config=peft_config,
        dataset_text_field="text",
        tokenizer=tokenizer,
        args=training_arguments,
)

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:245: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/2988 [00:00<?, ? examples/s]

Map:   0%|          | 0/996 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:317: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


### Testing before training

In [ ]:
# What are the reasons for the difference between the number of trainable parameters and the number of all parameters?
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = model.num_parameters()
    for _, param in model.named_parameters():
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

trainable params: 42520576 || all params: 7284252672 || trainable%: 0.583732853796316


In [ ]:
def prompt_model(model, tokenizer, eval_prompt, max_length=100):
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=max_length, pad_token_id=2)[0], skip_special_tokens=True))
    model.train()


In [ ]:
eval_prompt = """Once upon a time"""

prompt_model(model, tokenizer, eval_prompt)

In [ ]:
eval_prompt = """<s>[INST]What is the meaning of life?[/INST]"""

prompt_model(model, tokenizer, eval_prompt)

[INST]What is the meaning of life?[/INST]

[INST]What is the meaning of life?[/INST]

[INST]What is the meaning of life?[/INST]

[INST]What is the meaning of life?[/INST]

[INST]What is the meaning of life?[/INST]

[INST]What is the meaning of life?[/INST]

[INST]What is the meaning of life?[/INST]

[INST


In [ ]:
eval_prompt = """What is life?"""

prompt_model(model, tokenizer, eval_prompt)

What is life?

Life is a journey.

Life is a journey that we all take.

Life is a journey that we all take, and we all take it together.

Life is a journey that we all take, and we all take it together, and we all take it together, and we all take it together, and we all take it together, and we all take it together, and we all take it together, and we all take it together, and we all take


In [ ]:
eval_prompt = """[INST]<<SYS>>\nYou are a linguistic expert.\nYou will be given a TERM and QUESTION - ANSWER couple.\nThe QUESTION is composed with the TERM.\nIf the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".\nIf the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".\nIf the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". \nAnalyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.\nFirst reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.\n{"term":"",\n"reasoning":"",\n"certainty":""}\n<</SYS>>\nTERM => NASA Exoplanet Archive:The NASA Exoplanet Archive is an online astronomical exoplanet catalog and data service that collects and serves public data that support the search for and characterization of extra-solar planets (exoplanets) and their host stars. It is part of the Infrared Processing and Analysis Center and is on the campus of the California Institute of Technology (Caltech) in Pasadena, CA. The archive is funded by NASA and was launched in early December 2011 by the NASA Exoplanet Science Institute as part of NASA\'s Exoplanet Exploration Program. In July 2017, the archive\'s collection of confirmed exoplanets surpassed 3,500.\n\n\nQUESTION => How does the NASA Exoplanet Archive contribute to our understanding of the geodynamics of terrestrial exoplanets in the field of science and space exploration?\nANSWER => The NASA Exoplanet Archive plays a crucial role in advancing our understanding of the geodynamics of terrestrial exoplanets in the field of science and space exploration in several ways:\n\n1. Data Compilation: The Exoplanet Archive serves as a centralized repository for all publicly available data on exoplanets, including their physical properties, orbital characteristics, and host star information. By collecting and organizing this data, it provides scientists with a comprehensive resource to study the geodynamics of terrestrial exoplanets.\n\n2. Statistical Analysis: The Exoplanet Archive enables scientists to perform statistical analyses on the collected data. By studying the distribution of exoplanet properties, such as size, mass, and orbital parameters, researchers can identify trends and patterns that can shed light on the geodynamics of these planets. For example, the archive allows scientists to investigate the relationship between a planet\'s mass and its internal structure or the influence of stellar properties on the formation and evolution of exoplanets.\n\n3. Comparative Planetology: The Exoplanet Archive facilitates comparative studies between exoplanets and our own Solar System planets. By comparing the properties of exoplanets with those of Earth, Mars, or Venus, scientists can gain insights into the geodynamic processes occurring on these distant worlds. This comparative planetology approach helps in understanding the diversity of terrestrial planets and the factors that influence their geodynamics.\n\n4. Model Validation: The Exoplanet Archive provides observational data that can be used to validate and refine theoretical models of exoplanet geodynamics. Scientists can compare the predictions of their models with the observed properties of exoplanets to test the accuracy and reliability of their theories. This iterative process of model validation helps in improving our understanding of the geodynamics of terrestrial exoplanets.\n\n5. Target Selection: The Exoplanet Archive assists in the selection of target exoplanets for future missions and observations. By providing detailed information about the properties of known exoplanets, scientists can identify interesting candidates that are likely to provide valuable insights into the geodynamics of terrestrial planets. This helps in optimizing the allocation of resources for space exploration missions.\n\nOverall, the NASA Exoplanet Archive serves as a valuable resource for scientists studying the geodynamics of terrestrial exoplanets. It enables data compilation, statistical analysis, comparative planetology, model validation, and target selection, all of which contribute to advancing our understanding of these distant worlds and their geodynamic processes.\n\nINSTRUCTION => Output should be in the following json format:\n{"term":"NASA Exoplanet Archive",\n"reasoning":"",\n"certainty":""}\n[/INST]"""

prompt_model(model, tokenizer, eval_prompt)

[INST]<<SYS>>
You are a linguistic expert.
You will be given a TERM and QUESTION - ANSWER couple.
The QUESTION is composed with the TERM.
If the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".
If the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".
If the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". 
Analyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.
First reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.
{"term":"",
"reasoning":"",
"certainty":""}
<</SYS>>
TERM => NASA Exoplanet Archive:The NASA Exoplanet Archive is an online astronomical exoplanet catalog and data service that coll

In [ ]:
# Optional:Run trainer's default evaluator. How does it work? Why it is not suitable for our case? What could be a better evaluation?
# trainer.evaluate()

***** Running Evaluation *****
  Num examples = 5
  Batch size = 4


{'eval_loss': 1.137174367904663,
 'eval_runtime': 22.1582,
 'eval_samples_per_second': 0.226,
 'eval_steps_per_second': 0.09}

### Training

In [ ]:
# Launch the training
trainer.train()

Currently training with a batch size of: 4
***** Running training *****
  Num examples = 2,988
  Num Epochs = 1
  Instantaneous batch size per device = 4
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 1
  Total optimization steps = 747
  Number of trainable parameters = 42,520,576
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss


In [ ]:
# Run this cell while training continues not to lost your work if colab disconnects
trainer.push_to_hub(".../acceptance_evaluator_agent")

Saving model checkpoint to ./results
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
tokenizer config file saved in ./results/tokenizer_config.json
Special tokens file saved in ./results/special_tokens_map.json


adapter_model.safetensors:   0%|          | 0.00/694M [00:00<?, ?B/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

events.out.tfevents.1712021602.e23dcec5f38a.258.0:   0%|          | 0.00/5.65k [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/cemuluoglakci/results/commit/1e8bc1cda55cf0b1f7ce9c90f911537d7aae7b85', commit_message='cemuluoglakci/acceptance_evaluator_agent', commit_description='', oid='1e8bc1cda55cf0b1f7ce9c90f911537d7aae7b85', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
trainer.save_model("./my_model")

Saving model checkpoint to ./my_model
/usr/local/lib/python3.10/dist-packages/peft/utils/save_and_load.py:139: UserWarning: Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.
  warnings.warn("Setting `save_embedding_layers` to `True` as embedding layers found in `target_modules`.")
tokenizer config file saved in ./my_model/tokenizer_config.json
Special tokens file saved in ./my_model/special_tokens_map.json


### Testing after training

In [ ]:
# How did the behavior of the model changed? What are the advantages and disadvantages of supervised training?
# Regardless of our use case what alternative approaches can be taken to make LLMs follow instructions and prevent observed disadvantages while supervised training?

In [ ]:
# Optional:Run trainer's default evaluator.
# trainer.evaluate()

In [ ]:
eval_prompt = "Once upon a time"
model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=500, pad_token_id=2)[0], skip_special_tokens=True))
model.train()
eval_prompt = ""

Once upon a time, there was a linguistic expert.
You are a linguistic expert.
You will be given a TERM and QUESTION - ANSWER couple.
The QUESTION is composed with the TERM.
If the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".
If the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".
If the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". 
Analyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.
First reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.
{"term":"",
"reasoning":"",
"certainty":""}
<</br>
TERM => Dynamic visualization fusion: A fusion of dynamic visualization techniques, such as mot

In [ ]:
eval_prompt = """<s>[INST]What is the meaning of life?[/INST]"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))
model.train()
eval_prompt = ""

[INST]What is the meaning of life?[/INST]<<SYS>>
You are a linguistic expert.
You will be given a TERM and QUESTION - ANSWER couple.
The QUESTION is composed with the TERM.
If the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".
If the answer mentioned the term in any context even without any further information, definition, description or


In [ ]:
eval_prompt = """[INST]<<SYS>>\nYou are a linguistic expert.\nYou will be given a TERM and QUESTION - ANSWER couple.\nThe QUESTION is composed with the TERM.\nIf the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".\nIf the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".\nIf the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". \nAnalyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.\nFirst reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.\n{"term":"",\n"reasoning":"",\n"certainty":""}\n<</SYS>>\nTERM => NASA Exoplanet Archive:The NASA Exoplanet Archive is an online astronomical exoplanet catalog and data service that collects and serves public data that support the search for and characterization of extra-solar planets (exoplanets) and their host stars. It is part of the Infrared Processing and Analysis Center and is on the campus of the California Institute of Technology (Caltech) in Pasadena, CA. The archive is funded by NASA and was launched in early December 2011 by the NASA Exoplanet Science Institute as part of NASA\'s Exoplanet Exploration Program. In July 2017, the archive\'s collection of confirmed exoplanets surpassed 3,500.\n\n\nQUESTION => How does the NASA Exoplanet Archive contribute to our understanding of the geodynamics of terrestrial exoplanets in the field of science and space exploration?\nANSWER => The NASA Exoplanet Archive plays a crucial role in advancing our understanding of the geodynamics of terrestrial exoplanets in the field of science and space exploration in several ways:\n\n1. Data Compilation: The Exoplanet Archive serves as a centralized repository for all publicly available data on exoplanets, including their physical properties, orbital characteristics, and host star information. By collecting and organizing this data, it provides scientists with a comprehensive resource to study the geodynamics of terrestrial exoplanets.\n\n2. Statistical Analysis: The Exoplanet Archive enables scientists to perform statistical analyses on the collected data. By studying the distribution of exoplanet properties, such as size, mass, and orbital parameters, researchers can identify trends and patterns that can shed light on the geodynamics of these planets. For example, the archive allows scientists to investigate the relationship between a planet\'s mass and its internal structure or the influence of stellar properties on the formation and evolution of exoplanets.\n\n3. Comparative Planetology: The Exoplanet Archive facilitates comparative studies between exoplanets and our own Solar System planets. By comparing the properties of exoplanets with those of Earth, Mars, or Venus, scientists can gain insights into the geodynamic processes occurring on these distant worlds. This comparative planetology approach helps in understanding the diversity of terrestrial planets and the factors that influence their geodynamics.\n\n4. Model Validation: The Exoplanet Archive provides observational data that can be used to validate and refine theoretical models of exoplanet geodynamics. Scientists can compare the predictions of their models with the observed properties of exoplanets to test the accuracy and reliability of their theories. This iterative process of model validation helps in improving our understanding of the geodynamics of terrestrial exoplanets.\n\n5. Target Selection: The Exoplanet Archive assists in the selection of target exoplanets for future missions and observations. By providing detailed information about the properties of known exoplanets, scientists can identify interesting candidates that are likely to provide valuable insights into the geodynamics of terrestrial planets. This helps in optimizing the allocation of resources for space exploration missions.\n\nOverall, the NASA Exoplanet Archive serves as a valuable resource for scientists studying the geodynamics of terrestrial exoplanets. It enables data compilation, statistical analysis, comparative planetology, model validation, and target selection, all of which contribute to advancing our understanding of these distant worlds and their geodynamic processes.\n\nINSTRUCTION => Output should be in the following json format:\n{"term":"NASA Exoplanet Archive",\n"reasoning":"",\n"certainty":""}\n[/INST]"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=500, pad_token_id=2)[0], skip_special_tokens=True))
model.train()
eval_prompt = ""

[INST]<<SYS>>
You are a linguistic expert.
You will be given a TERM and QUESTION - ANSWER couple.
The QUESTION is composed with the TERM.
If the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".
If the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".
If the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". 
Analyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.
First reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.
{"term":"",
"reasoning":"",
"certainty":""}
<</SYS>>
TERM => NASA Exoplanet Archive:The NASA Exoplanet Archive is an online astronomical exoplanet catalog and data service that coll

In [ ]:
eval_prompt = """[INST]<<SYS>>\nYou are a linguistic expert.\nYou will be given a TERM and QUESTION - ANSWER couple.\nThe QUESTION is composed with the TERM.\nIf the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".\nIf the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".\nIf the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". \nAnalyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.\nFirst reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.\n{"term":"",\n"reasoning":"",\n"certainty":""}\n<</SYS>>\nTERM => Institute for Chemical-Physical Processes:IPCF is a multidisciplinary institute where theoretical, computational and experimental approaches are combined in specific fields of physical and chemical disciplines, namely new models and devices, environment and Soft Matter.\n\n\nQUESTION => How has the field of unconventional computing contributed to the advancement of technology, particularly in relation to emerging technologies like artificial intelligence and virtual reality, and how does it compare to the concept of Institute for Chemical-Physical Processes?\nANSWER => The field of unconventional computing has made significant contributions to the advancement of technology, including in relation to emerging technologies like artificial intelligence (AI) and virtual reality (VR). Unconventional computing explores alternative approaches to computation beyond traditional digital methods, often drawing inspiration from natural systems or physical phenomena. These approaches have the potential to enhance the capabilities and efficiency of computing systems.\n\nIn the context of AI, unconventional computing has explored bio-inspired algorithms and neural networks that mimic the behavior of biological systems. By leveraging the principles of neural computation, such as neural networks and evolutionary algorithms, unconventional computing has contributed to the development of more efficient and powerful AI algorithms. These algorithms have been applied to various AI applications, including image and speech recognition, natural language processing, and data analysis.\n\nSimilarly, in the realm of VR, unconventional computing has played a role in improving the immersive experience and computational efficiency. For instance, unconventional computing techniques have been used to develop algorithms that simulate complex physical interactions in virtual environments, leading to more realistic and interactive VR experiences. Additionally, unconventional computing has explored neuromorphic computing, which aims to replicate the brain\'s neural architecture, enabling faster and more efficient processing for VR applications.\n\nRegarding the concept of the Institute for Chemical-Physical Processes (ICPP), it is not directly comparable to unconventional computing. The ICPP is a research institution that focuses on studying chemical and physical processes, aiming to understand fundamental principles and develop applications based on these processes. While unconventional computing may draw inspiration from natural systems, including chemical and physical processes, it encompasses a broader range of computational paradigms beyond chemistry and physics.\n\nUnconventional computing encompasses various approaches, such as quantum computing, DNA computing, memristor-based computing, and more. These approaches explore unconventional substrates, architectures, and algorithms to perform computation. Their contributions to technology, including AI and VR, lie in their potential to overcome the limitations of traditional computing paradigms and enable novel computational capabilities.\n\nIn summary, unconventional computing has contributed to the advancement of technology, particularly in relation to emerging technologies like AI and VR. It has provided alternative computational paradigms and algorithms that enhance efficiency, mimic natural systems, and enable new capabilities. While the ICPP focuses on chemical and physical processes, unconventional computing encompasses a broader range of computational approaches and draws inspiration from various disciplines.\n\nINSTRUCTION => Output should be in the following json format:\n{"term":"Institute for Chemical-Physical Processes:IPCF is a multidisciplinary institute where theoretical, computational and experimental approaches are combined in specific fields of physical and chemical disciplines, namely new models and devices, environment and Soft Matter.\n",\n"reasoning":"",\n"certainty":""}\n[/INST]"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))
model.train()
eval_prompt = ""

[INST]<<SYS>>
You are a linguistic expert.
You will be given a TERM and QUESTION - ANSWER couple.
The QUESTION is composed with the TERM.
If the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".
If the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".
If the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". 
Analyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.
First reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.
{"term":"",
"reasoning":"",
"certainty":""}
<</SYS>>
TERM => Institute for Chemical-Physical Processes:IPCF is a multidisciplinary institute where theoretical, computational and exp

In [ ]:
eval_prompt = """[INST]<<SYS>>\nYou are a linguistic expert.\nYou will be given a TERM and QUESTION - ANSWER couple.\nThe QUESTION is composed with the TERM.\nIf the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".\nIf the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".\nIf the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". \nAnalyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.\nFirst reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.\n{"term":"",\n"reasoning":"",\n"certainty":""}\n<</SYS>>\nTERM => Calibration:In measurement technology and metrology, calibration is the comparison of measurement values delivered by a device under test with those of a calibration standard of known accuracy. Such a standard could be another measurement device of known accuracy, a device generating the quantity to be measured such as a voltage, a sound tone, or a physical artefact, such as a metre ruler.\n\n\nQUESTION => How does the concept of calibration, as applied in measurement technology and metrology, relate to the idea of fine-tuning compatibility within relationships, as proposed by the Compatibility (geochemistry)?\nANSWER => The concept of calibration in measurement technology and metrology is fundamentally different from the idea of fine-tuning compatibility within relationships proposed by the Compatibility (geochemistry).\n\nIn measurement technology and metrology, calibration refers to the process of comparing the measurements of an instrument or device to a known standard in order to determine its accuracy and reliability. It involves adjusting or aligning the instrument to ensure that it provides accurate and consistent measurements. Calibration is crucial in ensuring that measurements are traceable, reliable, and consistent across different instruments and laboratories.\n\nOn the other hand, the concept of fine-tuning compatibility within relationships, as proposed by the Compatibility (geochemistry), refers to the idea that certain elements or compounds are more likely to form stable and compatible combinations with each other. In geochemistry, compatibility is used to understand the behavior of elements during geological processes such as magma formation and crystallization. Elements that are considered compatible tend to coexist and form solid solutions, while incompatible elements may segregate or form separate minerals.\n\nWhile both concepts involve the idea of adjustment or alignment, they operate in different domains. Calibration in measurement technology focuses on ensuring accuracy and consistency in measurements, while fine-tuning compatibility in geochemistry focuses on understanding the behavior of elements in geological processes. The two concepts are not directly related to each other.\n\nINSTRUCTION => Output should be in the following json format:\n{"term":"Calibration:In measurement technology and metrology, calibration is the comparison of measurement values delivered by a device under test with those of a calibration standard of known accuracy. Such a standard could be another measurement device of known accuracy, a device generating the quantity to be measured such as a voltage, a sound tone, or a physical artefact, such as a metre ruler.\n",\n"reasoning":"",\n"certainty":""}\n[/INST]"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))
model.train()
eval_prompt = ""

[INST]<<SYS>>
You are a linguistic expert.
You will be given a TERM and QUESTION - ANSWER couple.
The QUESTION is composed with the TERM.
If the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".
If the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".
If the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". 
Analyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.
First reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.
{"term":"",
"reasoning":"",
"certainty":""}
<</SYS>>
TERM => Calibration:In measurement technology and metrology, calibration is the comparison of measurement values delivered by a 

In [ ]:
eval_prompt = """[INST]<<SYS>>\nYou are a linguistic expert.\nYou will be given a TERM and QUESTION - ANSWER couple.\nThe QUESTION is composed with the TERM.\nIf the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".\nIf the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".\nIf the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". \nAnalyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.\nFirst reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.\n{"term":"",\n"reasoning":"",\n"certainty":""}\n<</SYS>>\nTERM => Nano-Bio Organic Computing: A paradigm-shifting computing approach that combines nanotechnology and biotechnology, leveraging organic materials and processes at the nanoscale to create highly efficient and environmentally friendly computing systems.\n\nQUESTION => How does the field of nanobiomechanics contribute to the development and understanding of Nano-Bio Organic Computing in the realm of technology and gadgets?\nANSWER => The field of nanobiomechanics plays a crucial role in the development and understanding of Nano-Bio Organic Computing in the realm of technology and gadgets. Here are a few ways in which nanobiomechanics contributes to this field:\n\n1. Characterization of nanoscale materials: Nanobiomechanics involves studying the mechanical properties of biological materials at the nanoscale. This knowledge is essential for designing and engineering nanoscale components used in Nano-Bio Organic Computing. By understanding the mechanical behavior of biological materials, researchers can develop nanoscale devices that are compatible with biological systems.\n\n2. Biomimetic design: Nanobiomechanics draws inspiration from nature to design and fabricate nanoscale devices. By studying the mechanical properties of biological systems, researchers can mimic their structures and functions to create bio-inspired nanoscale components. These biomimetic designs are crucial for developing efficient and functional Nano-Bio Organic Computing devices.\n\n3. Integration of biological systems: Nanobiomechanics helps in integrating biological systems with nanoscale devices. By understanding the mechanical interactions between nanoscale components and biological systems, researchers can develop strategies to interface these systems effectively. This integration is essential for the development of Nano-Bio Organic Computing, where nanoscale devices interact with biological systems to perform computing tasks.\n\n4. Sensing and actuation: Nanobiomechanics contributes to the development of nanoscale sensors and actuators used in Nano-Bio Organic Computing. By understanding the mechanical properties of biological materials, researchers can design sensors that can detect and measure various biological signals. Additionally, nanobiomechanics helps in developing nanoscale actuators that can manipulate biological systems for computing purposes.\n\n5. Biocompatibility and safety: Nanobiomechanics also focuses on ensuring the biocompatibility and safety of nanoscale devices used in Nano-Bio Organic Computing. By studying the mechanical interactions between nanoscale components and biological systems, researchers can identify potential risks and develop strategies to minimize them. This is crucial for the successful integration of nanoscale devices into biological systems without causing harm.\n\nOverall, nanobiomechanics provides the foundational knowledge and techniques required for the development and understanding of Nano-Bio Organic Computing. It enables the design of nanoscale components, integration with biological systems, development of sensors and actuators, and ensures biocompatibility and safety, all of which are essential for advancing this field of technology and gadgets.\n\nINSTRUCTION => Output should be in the following json format:\n{"term":"Nano-Bio Organic Computing: A paradigm-shifting computing approach that combines nanotechnology and biotechnology, leveraging organic materials and processes at the nanoscale to create highly efficient and environmentally friendly computing systems.",\n"reasoning":"",\n"certainty":""}\n[/INST]"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100, pad_token_id=2)[0], skip_special_tokens=True))
model.train()
eval_prompt = ""

[INST]<<SYS>>
You are a linguistic expert.
You will be given a TERM and QUESTION - ANSWER couple.
The QUESTION is composed with the TERM.
If the answer claims that the term is unreal, hypothetical, speculative or do not exist, label the certainty of the term as "UNREAL".
If the answer mentioned the term in any context even without any further information, definition, description or explanation, label the certainty of the term as "MENTIONED".
If the answer states that it does not have information about the term , label the certainty of the term as "UNKNOWN". 
Analyse the certainty of the term only by itself. Do not consider its relation with other terms or the context of the answer.
First reason step by step then label certainty of the term as "UNREAL", "MENTIONED" or "UNKNOWN"  in the following json format.
{"term":"",
"reasoning":"",
"certainty":""}
<</SYS>>
TERM => Nano-Bio Organic Computing: A paradigm-shifting computing approach that combines nanotechnology and biotechnology, lever